In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


In [2]:
img_width, img_height = 256, 256
train_data_dir = r"./kaggleDataset/trainData/"
nb_train_steps = 153 
nb_validation_steps = 8 
batch_size = 64
epochs = 100
num_classes = 120

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))


# Freeze all the layers in model.layers:
    layer.trainable = False

In [3]:
x = model.output
x = Flatten()(x)
x = Dropout(0.4)(x)
# Adding 2 fully-connected layers
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [4]:
# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [5]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.adam(lr=0.001), metrics=["accuracy"])


In [6]:
# Initiate the train and test generators with data Augumentation 
datagen = ImageDataGenerator(
rescale = 1./255,
#horizontal_flip = True,
#fill_mode = "nearest",
#zoom_range = 0.1,
#width_shift_range = 0.1,
#height_shift_range=0.1,
#rotation_range=30,
validation_split=0.05)

train_generator = datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
color_mode="rgb",
batch_size=32,
class_mode="categorical",
shuffle=True,
seed=42,
subset='training')

validation_generator = datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
color_mode="rgb",
batch_size=32,
class_mode='categorical',
subset='validation')


Found 9745 images belonging to 120 classes.
Found 453 images belonging to 120 classes.


In [7]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("./TLModels/Ori_vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
tbCallBack = keras.callbacks.TensorBoard(log_dir='./TLlogs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)


In [8]:
# Train the model
model_final.load_weights("./TLModels/vgg16_1.h5")

model_final.fit_generator(
train_generator,
steps_per_epoch = nb_train_steps,
epochs = epochs,
validation_data = validation_generator,
validation_steps = nb_validation_steps,
callbacks = [checkpoint,tbCallBack])

Epoch 1/100
153/153 [==============================] - 63s 411ms/step - loss: 1.0177 - acc: 0.7081 - val_loss: 3.3099 - val_acc: 0.2852

Epoch 00001: val_acc improved from -inf to 0.28516, saving model to ./TLModels/Ori_vgg16_1.h5
Epoch 2/100
153/153 [==============================] - 61s 396ms/step - loss: 1.1163 - acc: 0.6756 - val_loss: 3.4709 - val_acc: 0.2271

Epoch 00002: val_acc did not improve from 0.28516
Epoch 3/100
153/153 [==============================] - 60s 391ms/step - loss: 0.7632 - acc: 0.7772 - val_loss: 3.4590 - val_acc: 0.2969

Epoch 00003: val_acc improved from 0.28516 to 0.29688, saving model to ./TLModels/Ori_vgg16_1.h5
Epoch 4/100
153/153 [==============================] - 59s 388ms/step - loss: 0.8492 - acc: 0.7476 - val_loss: 3.5977 - val_acc: 0.2795

Epoch 00004: val_acc did not improve from 0.29688
Epoch 5/100
153/153 [==============================] - 60s 390ms/step - loss: 0.5947 - acc: 0.8268 - val_loss: 3.6159 - val_acc: 0.2734

Epoch 00005: val_acc did

153/153 [==============================] - 60s 391ms/step - loss: 0.1244 - acc: 0.9625 - val_loss: 5.1772 - val_acc: 0.2852

Epoch 00086: val_acc did not improve from 0.32751
Epoch 87/100
153/153 [==============================] - 60s 391ms/step - loss: 0.1254 - acc: 0.9608 - val_loss: 5.3309 - val_acc: 0.2227

Epoch 00087: val_acc did not improve from 0.32751
Epoch 88/100
153/153 [==============================] - 60s 392ms/step - loss: 0.1248 - acc: 0.9610 - val_loss: 5.5509 - val_acc: 0.2695

Epoch 00088: val_acc did not improve from 0.32751
Epoch 89/100
153/153 [==============================] - 60s 391ms/step - loss: 0.1425 - acc: 0.9571 - val_loss: 5.3453 - val_acc: 0.2926

Epoch 00089: val_acc did not improve from 0.32751
Epoch 90/100
153/153 [==============================] - 60s 390ms/step - loss: 0.1358 - acc: 0.9579 - val_loss: 5.4437 - val_acc: 0.2183

Epoch 00090: val_acc did not improve from 0.32751
Epoch 91/100
153/153 [==============================] - 60s 393ms/step - 